In [1]:
import sys 
from pathlib import Path

module_path = str(Path.cwd().parents[0])
sys.path.append(module_path)

___

In [2]:
# from src.models.mymodel import model
from src.example_function import f

In [3]:
f(200)

201

In [4]:
from src.data_generator import DataGenerator

2023-05-16 12:06:56.371287: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-16 12:06:56.371310: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [5]:
params = {'dim': (2,525),
          'batch_size': 128,
          'n_classes': 10, #shouldn't be categorical
          'n_channels': 4,
          'shuffle': True}

## Usage of DataGenerator

In [6]:
import random
import numpy as np

from Bio import AlignIO
from collections import defaultdict

In [7]:
# 1. load alignment
file_name = "../test_aln_na.fa"
file_format = 'fasta'

aln = AlignIO.read(file_name, file_format)
print(aln)

Alignment with 29 rows and 525 columns
ATGCAAAAGGTAAAACTACCCCTCACTATTGACCCATATAAAGA...AAC NC_009053|APL_RS07335
ATGCAAAAGGTAAAACTACCCCTCACTATTGACCCATATAAAGA...AAC NZ_CP026009|CYK57_RS07840
ATGCAAAAGGTAAAACTACCCCTCACTATTGACCCATATAAAGA...AAC NZ_CP063424|IOD66_RS07630
ATGCAAAAGGTAAAACTACCCCTCACTATTGACCCATATAAAGA...AAC NZ_CP069797|D1098_RS07375
ATGCAAAAGGTAAAACTACCCCTCACCATTGACCCATATAAAGA...AAC NC_010278|APJL_RS07040
ATGCAAAAGGTAAAACTACCCCTCACCATTGACCCATATAAAGA...AAC NC_010939|APP7_RS07425
ATGCAAAAGGTAAAACTACCCCTCACCATTGACCCATATAAAGA...AAC NZ_CP022715|CHY23_RS02125
ATGCAAAAGGTAAAACTACCCCTCACCATTGACCCATATAAAGA...AAC NZ_CP029003|APPSER1_RS07590
ATGCAAAAGGTAAAACTACCCCTCACCATTGACCCATATAAAGA...AAC NZ_CP030753|DRF63_RS07590
ATGCAAAAGGTAAAACTACCCCTCACTATTGACCCATATAAAGA...AAC NZ_CP031858|D1109_RS04885
ATGCAAAAGGTAAAACTACCCCTCACCATTGACCCATATAAAGA...AAC NZ_CP031859|D1108_RS07025
ATGCAAAAGGTAAAACTACCCCTCACCATTGACCCATATAAAGA...AAC NZ_CP031861|D1106_RS07350
ATGCAAAAGGTAAAACTACCCCTCACCATTGACCCATATAAAGA.

In [8]:
# dict to create one-hot encoding of sequences
hot_dict = {'A' : np.array([0, 0, 0, 1]),
           'T' : np.array([0, 0, 1, 0]),
           'G' : np.array([0, 1, 0, 0]),
           'C' : np.array([1, 0, 0, 0]),
           '-' : np.array([0, 0, 0, 0]),}


In [9]:
# encode sequences as one-hot
encoded_seqs = [np.array([hot_dict[car] for car in allignment.seq]) for allignment in aln]

In [10]:
# creating fake distances in the tree
nodes = range(29)
tree_dist = defaultdict(dict)
for node1 in nodes: 
    for node2 in nodes: 
        tree_dist[node1][node2] = random.random()


In [11]:
tree_dist      

defaultdict(dict,
            {0: {0: 0.14325626006203152,
              1: 0.22895821478422762,
              2: 0.8214952166248951,
              3: 0.07932094995864092,
              4: 0.870163086730506,
              5: 0.5877891529849268,
              6: 0.19470036047260197,
              7: 0.8301452899514158,
              8: 0.2991139225207612,
              9: 0.35969575757276984,
              10: 0.6331943298974761,
              11: 0.3038194866347823,
              12: 0.8505487653601944,
              13: 0.1092215262678542,
              14: 0.6932172512666319,
              15: 0.5387410066140345,
              16: 0.27306774053214555,
              17: 0.5033220172762052,
              18: 0.5377757372534929,
              19: 0.3424771872540944,
              20: 0.8724473521585221,
              21: 0.7946907179576879,
              22: 0.7745032640852183,
              23: 0.772826355665064,
              24: 0.07905594215791556,
              25: 0.34556062564919

In [12]:
model_inputs = [
                 [(encoded_seqs[i], encoded_seqs[j], tree_dist[i][j]) for j in tree_dist[i] if j <= 28]
                for i in tree_dist if i <= 28
             ]

In [13]:
type(model_inputs[0]), len(model_inputs[0])


(list, 29)

In [14]:
seq1, seq2, distance = model_inputs[0][0]
print(distance)

0.14325626006203152


In [15]:
lista_IDs_train = [[i, j] for j in range(0, 22) for i in range(0, 22)]
lista_IDs_val = [[i, j] for j in range(22, 29) for i in range(22, 29)]

In [16]:
train_ds_gen = DataGenerator(lista_IDs_train, model_inputs=model_inputs, **params)
# test_ds_gen = DataGenerator(lista_IDs_val, **params)


In [18]:
train_iter = iter(train_ds_gen)

In [20]:
batch = next(train_iter)

In [22]:
type(batch), len(batch)

(tuple, 2)

In [23]:
t1, t2 = batch

In [25]:
t1.shape

(128, 2, 525, 4)

In [ ]:
params = {'dim': (2,525),
          'batch_size': 128,
          'n_classes': 10, #shouldn't be categorical
          'n_channels': 4,
          'shuffle': True}